## Prompting

One of the few resources found to Prefix Templates with OpenPrompt

> https://github.com/thunlp/OpenPrompt/blob/main/tutorial/2.1_conditional_generation.py

### Imports

In [ ]:
! pip install -q openprompt transformers dataset

In [ ]:
from openprompt import PromptDataLoader, PromptForGeneration
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt.prompts import PrefixTuningTemplate
from openprompt.utils.metrics import generation_metric
from sklearn.model_selection import train_test_split
from datasets.dataset_dict import DatasetDict, Dataset
# from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from tqdm import tqdm
from transformers.optimization import get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
import pandas as pd
from pathlib import Path


### Variables

In [3]:
base_path = "./"
csv_data = "./datasets/df_songs.csv"
used_model = "gpt2"

train_split = 0.7
epochs = 7
batch_size = 2

### Model / Data Preparation

#### Dataset

Read the CSV, remove everything except the lyrics. Add index for flavour.

In [4]:
lyrics_df = pd.read_csv(csv_data)
lyrics_df = lyrics_df.drop(
    columns=["Artist", "Song", "LyricsWordCount"], errors="ignore"
).reset_index(level=0)


Split the dataset and create an DatasetDict

In [5]:
train_df, validation_df = train_test_split(lyrics_df, train_size=train_split)
train_dataset, validation_dataset = Dataset.from_pandas(train_df), Dataset.from_pandas(
    validation_df
)
raw_dataset = DatasetDict({"train": train_dataset, "validation": validation_dataset})


Create a new dataset with a mapped `InputExample` for each sample

In [6]:
dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    for data in raw_dataset[split]:
        # input_example = InputExample(text_a = data['premise'], text_b = data['hypothesis'], label=int(data['label']), guid=data['idx'])
        input_example = InputExample(text_a = data['Lyrics'], guid=data['index'])
        dataset[split].append(input_example)

Steal this dataloader wrapper function 🐱‍👤

In [7]:
def get_dataloader(
    dataset_split, template, tokenizer, wrapper_class, shuffle=False, batch_size=32
):
    """Returns a prompt data load for a given dataset split and template"""

    return PromptDataLoader(
        dataset=dataset_split,
        template=template,
        tokenizer=tokenizer,
        tokenizer_wrapper_class=wrapper_class,
        max_seq_length=256,
        decoder_max_length=256,
        batch_size=batch_size,
        shuffle=shuffle,
        teacher_forcing=False,
        predict_eos_token=True,
        truncate_method="head",
    )


#### Model (PLM)

In [8]:
plm, tokenizer, model_config, WrapperClass = load_plm(used_model, used_model)


Using pad_token, but it is not set yet.


In [9]:
# # tokenizer = GPT2Tokenizer.from_pretrained(used_model, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

# tokenizer.bos_token = "<|startoftext|>"
# tokenizer.eos_token = "<|endoftext|>"
# tokenizer.pad_token = "<|pad|>"


### Prompt-Based Fine-Tuning

Create a template.
The used template (line 1) equals the last template (line 7), so that the text param can be omitted.

In [10]:
template = PrefixTuningTemplate(model=plm, tokenizer=tokenizer, text=' {"placeholder":"text_a"} {"special": "<eos>"} {"mask"} ', using_decoder_past_key_values=False)

# Are the tokens necessary? Probably not:
# # You may observe that the example doesn't end with <|endoftext|> token. Don't worry, adding specific end-of-text token
# # is a language-model-specific token. we will add it for you in the TokenizerWrapper once you pass `predict_eos_token=True`

# template = PrefixTuningTemplate(model=plm, tokenizer=tokenizer, text='<|startoftext|>{"placeholder":"text_a"} {"mask"}<|endoftext|>')

Ignore using_decoder_past_key_values=False in a decoder-only LM.


Create one example and print it, to see what it looks like.

In [11]:
print(template.wrap_one_example(dataset['train'][0]))

[[{'text': ' Snoopy\nIs what they say as if they knew me\nGroupies on my head like a cufi\nMy nigga Kid Cudi thats my little buddy\nCall some hoes up and get some cutty cutty\nWhats your life like Mines is kinda tight\nA long way from hustlin that china white\nMy people love me the fans love me Im all go\nIf you aint showin love then what you call for\nI dont need it in my life my kids pay the price\nSo alike and in need of my wife\nSmoke til Im gone Dogg on the throne\nIm worldwide known Im a boss in the zone\nI get it how I get it cause I can I earn my spot\nYou see the plan Im just fuckin with my fans\nI turned out to be the better man important to life\nYou understand while Im puffin on this gram\nEverything Im havin no there aint necessity\nThough Im shining keep on grindin\nWhat you see aint all of me\nThough I keep them hos dont love them hos\nThe code in which yall roll\nIts so simple what I need\nYou know I keep my fam and I cant forget that tree\nNah nah nah nah nah nah\nI ca

In [12]:
train_dataloader = get_dataloader(
    dataset["train"],
    template,
    tokenizer,
    WrapperClass,
    shuffle=True,
    batch_size=batch_size,
)
validation_dataloader = get_dataloader(
    dataset["validation"],
    template,
    tokenizer,
    WrapperClass,
    shuffle=False,
    batch_size=batch_size,
)


tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1113 > 1024). Running this sequence through the model will result in indexing errors
tokenizing: 5714it [00:13, 431.26it/s]
tokenizing: 2449it [00:05, 466.83it/s]


In [13]:
prompt_model = PromptForGeneration(plm=plm,template=template, freeze_plm=True,tokenizer=tokenizer)

In [14]:
# Follow PrefixTuning（https://github.com/XiangLi1999/PrefixTuning), we also fix the language model
# only include the template's parameters in training.

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in template.named_parameters()
            if (not any(nd in n for nd in no_decay)) and p.requires_grad
        ],
        "weight_decay": 0.0,
    },
    {
        "params": [
            p
            for n, p in template.named_parameters()
            if any(nd in n for nd in no_decay) and p.requires_grad
        ],
        "weight_decay": 0.0,
    },
]

optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)


In [15]:
tot_step  = len(train_dataloader)*5
scheduler = get_linear_schedule_with_warmup(optimizer, 0, tot_step)

In [16]:
generation_arguments = {
    "max_length": 512,
    "max_new_tokens": None,
    "min_length": 5,
    "temperature": 1.0,
    "do_sample": False,
    "top_k": 0,
    "top_p": 0.9,
    "repetition_penalty": 1.0,
    "num_beams": 5,
    "bad_words_ids": [[628], [198]]
}

def evaluate(prompt_model, dataloader):
    generated_sentence = []
    groundtruth_sentence = []
    prompt_model.eval()

    for step, inputs in enumerate(dataloader):
        _, output_sentence = prompt_model.generate(inputs, **generation_arguments)
        generated_sentence.extend(output_sentence)
        groundtruth_sentence.extend(inputs['tgt_text'])
    score = generation_metric(generated_sentence, groundtruth_sentence, "sentence_bleu")
    print("test_score", score, flush=True)
    return generated_sentence

In [17]:
# training and generation.
global_step = 0
tot_loss = 0
log_loss = 0
for epoch in range(epochs):
    prompt_model.train()
    for step, inputs in tqdm(enumerate(train_dataloader)):
        global_step +=1
        loss = prompt_model(inputs)
        loss.backward()
        tot_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(template.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if global_step %500 ==0:
            print("Epoch {}, global_step {} average loss: {} lr: {}".format(epoch, global_step, (tot_loss-log_loss)/500, scheduler.get_last_lr()[0]), flush=True)
            log_loss = tot_loss

generated_sentence = evaluate(prompt_model, validation_dataloader)

15it [10:41, 42.80s/it]

In [ ]:
with open(base_path + "generated_sentences.txt",'w') as f:
    for i in generated_sentence:
        f.write(i+"\n")